모델은 반드시 gpt-2 데이터 다르게

# SCRIPT

## Import Library and Config

### library

In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer, set_seed
from datasets import load_dataset
from tqdm import tqdm
import json
import torch
import argparse
from datetime import date, datetime
from os import listdir
from itertools import chain

/rds/general/user/jj1122/home/anaconda3/envs/m2d2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### config

In [15]:
# model_path_format = "/rds/general/user/jj1122/home/projects/m2d2/dataset/{model_id}/models"
# ckpt_path_format = "/checkpoint-{ckpt}"
cache_dir = "/rds/general/user/jj1122/ephemeral/.cache/huggingface"
# General_referece
# list_of_datasets = [
#     "cs_l1",
#     "nlin_l1",
#     "Health_and_fitness",
#     "Natural_and_physical_sciences",
#     "Religion_and_belief_systems",
#     "Culture_and_the_arts",
#     "General_referece",
#     "econ_l1",
#     "History_and_events",
#     "Human_activites",
#     "Mathematics_and_logic",
# #     "astro-ph_l1",
# #     "cond-mat_l1",
#     "eess_l1",
# #     "math_l1",
# #     "physics_l1 (ERROR)",
#     "q-bio_l1",
#     "q-fin_l1",
#     "stat_l1",
#     "Philosophy",
#     "Philosophy_and_thinking",
#     "Society_and_social_sciences",
#     "Technology_and_applied_sciences",

# ]

device_id = "cuda"
today_dt = date.today().strftime("%y%m%d")

model_id = "gpt2"
# model_type = "zsh"

# output_file = f"./output_logs/{today_dt}_{model_type}.json"
# ds = "Religion_and_belief_systems"
# ds = "econ_l1"
ds = "physics_l1"
# General_referece
model_path = f"/rds/general/user/jj1122/home/projects/m2d2/dataset/{ds}/models"

# econ_l1
stride = 1024

## Load Model, Tokeniser, Dataset

In [18]:
set_seed(718)
# model = GPT2LMHeadModel.from_pretrained(model_path, cache_dir=cache_dir).to(device_id)
# model.eval()
# tokenizer = GPT2TokenizerFast.from_pretrained(model_path)

dataset = load_dataset(
    "machelreid/m2d2", ds, cache_dir=cache_dir
)

Found cached dataset m2d2 (/rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/physics_l1/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00)
100%|██████████| 3/3 [00:05<00:00,  1.97s/it]


## Tokenisation & Grouping Text

In [17]:
dict_input_ids = {}

for gubun in ["validation"]: # ["train", "validation", "test"]:
    if gubun == "train": continue

    # TOKENISATION
    encodings = tokenizer("\n".join(dataset[gubun]["text"]), return_tensors="pt")

    # GROUPING TEXT
    with torch.no_grad():
        max_length = model.config.n_positions
        nlls = []
        list_input_ids = []
        for i in tqdm(range(0, encodings.input_ids.size(1), stride)):
            begin_loc = max(i + stride - max_length, 0)
            end_loc = min(i + stride, encodings.input_ids.size(1))
            trg_len = end_loc - i  # may be different from stride on last loop
            input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device_id)
            target_ids = input_ids.clone()
            target_ids[:, :-trg_len] = -100
            list_input_ids.append(input_ids)


100%|██████████| 2395/2395 [00:00<00:00, 8285.61it/s]


In [1]:
# dataset[gubun]["text"]

# Results
## Raw Dataset

In [20]:
dataset["validation"]["text"][:5]

['In full agreement with the results of the new quantum theory directional quantization appears as a general and universal feature of quantum measurements. One experimental example for such directional quantization in scattering processes is shown. Last not least, the early history of the "almost" discovery of the electron spin in the SGE is revisited.',
 "In almost all introductory textbooks on atomic physics the Stern-Gerlach experiment (SGE), performed by Otto Stern (1888 Stern ( -1969 and Walther Gerlach (1889 -1979 in Frankfurt in 1922 (Stern, 1920a; Gerlach and Stern, 1921 , 1922b Gerlach, 1925) , is presented as a benchmark experiment of quantum science. In most textbooks, the SGE is taken as evidence for proving that Pieter Debye's (Debye, 1916) and Arnold Sommerfeld's (Sommerfeld, 1916) hypothesis of directional quantization of magnetic and electric momenta of quantum objects in the presence of electric and magnetic fields is a real fact in the quantum world and that magnetic 

## Tokenisation

In [21]:
print(encodings['input_ids'][0][:20])

tensor([23799,    41,    57,    43,   220, 13695,   270,  1757,   366,    51,
        13411,     1,  4373,   357,  6286,   807,  3035,  9656,   828,  1365])


In [11]:
print(tokenizer.convert_ids_to_tokens(encodings['input_ids'][0][:20]))

['ĠTB', 'J', 'Z', 'L', 'Ġ', 'ĠTob', 'it', 'ĠJohn', 'Ġ"', 'T', 'obi', '"', 'ĠBrown', 'Ġ(', 'born', 'Ġ8', 'ĠApril', 'Ġ1993', '),', 'Ġbetter']


In [12]:
len(encodings['input_ids'][0])

2451922

In [13]:
len([x for x in list(chain(*(encodings['input_ids']))) if x not in [198, 628] ])

2451922

In [14]:
len([x for x in list(chain(*(encodings['input_ids']))) if x in [198, 628] ])

0

## Grouping Text

In [236]:
list_input_ids[0][:10]

tensor([[  198, 22737,    41,  ...,   587,  1444,   262]], device='cuda:0')

In [237]:
print(tokenizer.convert_ids_to_tokens(list_input_ids[0][0][:10]))

['Ċ', 'TB', 'J', 'Z', 'L', 'Ċ', 'Ċ', 'T', 'ob', 'it']


In [238]:
len(list_input_ids)

2443

# Script to Trainer pre-processing

## Tokenisation

In [239]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1394404
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 55451
    })
    test: Dataset({
        features: ['text'],
        num_rows: 55451
    })
})

In [240]:
def tokenize_function(examples):
    output = tokenizer(examples['text'])
    # clm input could be much much longer than block_size
    return output
    
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=8,
    remove_columns=['text'],
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)


Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/General_referece/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-c48f7bc4f6e010f1_*_of_00008.arrow
Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/General_referece/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-e6a697a7f547c421_*_of_00008.arrow
Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/General_referece/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-4023b72209bfaf59_*_of_00008.arrow


In [241]:
len(list(chain(*(tokenized_datasets['validation']['input_ids']))))

2449110

In [242]:
len([x for x in list(chain(*(tokenized_datasets['validation']['input_ids']))) if x == 198 ])

0

## group_texts

In [243]:
block_size = tokenizer.model_max_length
if block_size > 1024:
    block_size = 1024

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=8,
    load_from_cache_file=True,
    desc=f"Grouping texts in chunks of {block_size}",
)

Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/General_referece/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-f04adc3e638b2ad4_*_of_00008.arrow
Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/General_referece/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-1fbdd863da8a8924_*_of_00008.arrow
Loading cached processed dataset at /rds/general/user/jj1122/ephemeral/.cache/huggingface/machelreid___m2d2/General_referece/0.0.0/eb235f33a5de3163c10549b7f63c906910539c8a8c0ec5ade1285ccbf5067d00/cache-daadfba2f078454b_*_of_00008.arrow


## Results

### Raw dataset

In [244]:
dataset["validation"]["text"][:5]

['',
 'TBJZL',
 '',
 'Tobit John "Tobi" Brown (born 8 April 1993), better known as TBJZL, is an English YouTuber, streamer, Internet personality, and rapper. He is also a member and co-founder of the British YouTube group known as the Sidemen.',
 'In 2019, Brown was listed as the 38th most influential online creator in the United Kingdom by "The Sunday Times". , his main YouTube channel has over 4 million subscribers and 400 million video views. Brown has also released music mononymously as Tobi, and in 2020, he released his debut single "Destined for Greatness", which reached number 31 on the UK Singles Chart.']

### Tokenisation

In [245]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1394404
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 55451
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 55451
    })
})

In [246]:
print(tokenized_datasets["validation"]["input_ids"][:5])

[[], [22737, 41, 57, 43], [], [51, 672, 270, 1757, 366, 51, 13411, 1, 4373, 357, 6286, 807, 3035, 9656, 828, 1365, 1900, 355, 23799, 41, 57, 43, 11, 318, 281, 3594, 921, 51, 18478, 11, 4269, 263, 11, 4455, 8806, 11, 290, 25670, 13, 679, 318, 635, 257, 2888, 290, 763, 12, 15454, 286, 262, 3517, 7444, 1448, 1900, 355, 262, 15686, 8952, 13], [818, 13130, 11, 4373, 373, 5610, 355, 262, 4353, 400, 749, 14212, 2691, 13172, 287, 262, 1578, 7526, 416, 366, 464, 3502, 3782, 1911, 837, 465, 1388, 7444, 6518, 468, 625, 604, 1510, 18327, 290, 7337, 1510, 2008, 5009, 13, 4373, 468, 635, 2716, 2647, 937, 5177, 3481, 355, 309, 13411, 11, 290, 287, 12131, 11, 339, 2716, 465, 8886, 2060, 366, 24159, 1389, 329, 3878, 1108, 1600, 543, 4251, 1271, 3261, 319, 262, 3482, 5573, 829, 22086, 13]]


In [247]:
print(tokenizer.convert_ids_to_tokens(chain(*tokenized_datasets["validation"]["input_ids"][:5])))

['TB', 'J', 'Z', 'L', 'T', 'ob', 'it', 'ĠJohn', 'Ġ"', 'T', 'obi', '"', 'ĠBrown', 'Ġ(', 'born', 'Ġ8', 'ĠApril', 'Ġ1993', '),', 'Ġbetter', 'Ġknown', 'Ġas', 'ĠTB', 'J', 'Z', 'L', ',', 'Ġis', 'Ġan', 'ĠEnglish', 'ĠYou', 'T', 'uber', ',', 'Ġstream', 'er', ',', 'ĠInternet', 'Ġpersonality', ',', 'Ġand', 'Ġrapper', '.', 'ĠHe', 'Ġis', 'Ġalso', 'Ġa', 'Ġmember', 'Ġand', 'Ġco', '-', 'founder', 'Ġof', 'Ġthe', 'ĠBritish', 'ĠYouTube', 'Ġgroup', 'Ġknown', 'Ġas', 'Ġthe', 'ĠSid', 'emen', '.', 'In', 'Ġ2019', ',', 'ĠBrown', 'Ġwas', 'Ġlisted', 'Ġas', 'Ġthe', 'Ġ38', 'th', 'Ġmost', 'Ġinfluential', 'Ġonline', 'Ġcreator', 'Ġin', 'Ġthe', 'ĠUnited', 'ĠKingdom', 'Ġby', 'Ġ"', 'The', 'ĠSunday', 'ĠTimes', '".', 'Ġ,', 'Ġhis', 'Ġmain', 'ĠYouTube', 'Ġchannel', 'Ġhas', 'Ġover', 'Ġ4', 'Ġmillion', 'Ġsubscribers', 'Ġand', 'Ġ400', 'Ġmillion', 'Ġvideo', 'Ġviews', '.', 'ĠBrown', 'Ġhas', 'Ġalso', 'Ġreleased', 'Ġmusic', 'Ġmon', 'onym', 'ously', 'Ġas', 'ĠT', 'obi', ',', 'Ġand', 'Ġin', 'Ġ2020', ',', 'Ġhe', 'Ġreleased', 'Ġhis', 'Ġd

In [248]:
list_token_diff = set(encodings['input_ids'][0].tolist()) \
    - set(list(chain(*tokenized_datasets["validation"]["input_ids"])))


In [249]:
list_token_diff

{198, 628}

In [250]:
len(list_token_diff)

2

In [251]:
tokenizer.convert_ids_to_tokens([198, 628])

['Ċ', 'ĊĊ']

In [252]:
len(list(chain(*tokenized_datasets["validation"]["input_ids"])))

2449110

### Grouping Text

In [253]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 76543
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2364
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2290
    })
})

In [254]:
lm_datasets['validation'][0]['input_ids'][:5]

[22737, 41, 57, 43, 51]

In [255]:
print(tokenizer.convert_ids_to_tokens(lm_datasets['validation'][0]['input_ids'][:5]))


['TB', 'J', 'Z', 'L', 'T']


In [256]:
len(lm_datasets['validation'])

2364

# Perplexity with Trainer pre-processing

In [257]:
nlls = []
def eval_ppl_v3(model, tokenized_datasets, stride, device=device_id):
    max_length = model.config.n_positions
    nlls = []
    encodings = list(chain(*(tokenized_datasets['validation']['input_ids'])))
    n_words = len(encodings)

    encodings = torch.tensor([encodings])
    
    print(len(list(range(0, n_words, stride))))
    for i in tqdm(range(0, n_words, stride)):
        begin_loc = max(i + stride - max_length, 0)
        end_loc = min(i + stride, n_words)
        trg_len = end_loc - i  # may be different from stride on last loop
        # input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        input_ids = encodings[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            neg_log_likelihood = outputs[0] * trg_len

        nlls.append(neg_log_likelihood)

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl

In [258]:
with torch.no_grad():
    ppl = eval_ppl_v3(model, tokenized_datasets, stride=1024).item()


2392


100%|██████████| 2392/2392 [01:28<00:00, 27.16it/s]


In [259]:
len(nlls)

0

In [260]:
ppl

22.250993728637695

In [34]:
# encodings format 한번더 확 인
encodings = list(chain(*tokenized_datasets["validation"]["input_ids"]))

In [35]:
!pwd

/rds/general/user/jj1122/home/projects/domain_adaptation_m2d2
